### Importing Libraries

In [71]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [72]:
base_path = "/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition"
lines_list = []

lines = open(f"{base_path}/lines.txt", "r").readlines()
for line in lines:
    lines_list.append(line)

len(lines_list)
np.random.shuffle(lines_list)


In [73]:
!head -20 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines.txt'

a01-000u-00 ok 154 19 408 746 1661 89 A|MOVE|to|stop|Mr.|Gaitskell|from
a01-000u-01 ok 156 19 395 932 1850 105 nominating|any|more|Labour|life|Peers
a01-000u-02 ok 157 16 408 1106 1986 105 is|to|be|made|at|a|meeting|of|Labour
a01-000u-03 err 156 23 430 1290 1883 70 M Ps|tomorrow|.|Mr.|Michael|Foot|has
a01-000u-04 ok 157 20 395 1474 1830 94 put|down|a|resolution|on|the|subject
a01-000u-05 err 156 21 379 1643 1854 88 and|he|is|to|be|backed|by|Mr.|Will
a01-000u-06 ok 159 20 363 1825 2051 87 Griffiths|,|M P|for|Manchester|Exchange|.
a01-000x-00 ok 182 30 375 748 1561 148 A|MOVE|to|stop|Mr.|Gaitskell|from|nominating
a01-000x-01 ok 181 23 382 924 1595 148 any|more|Labour|life|Peers|is|to|be|made|at|a
a01-000x-02 ok 181 30 386 1110 1637 140 meeting|of|Labour|0M Ps|tomorrow|.|Mr.|Michael
a01-000x-03 ok 179 22 375 1276 1584 154 Foot|has|put|down|a|resolution|on|the|subject
a01-000x-04 ok 173 25 397 1458 1647 148 and|he|is|to|be|backed|by|Mr.|Will|Griffiths|,
a01-000x-05 ok 173 16 393 1635 1082 

In [74]:
len(lines_list)

13353

Splitting the dataset into train test and validation (90:5:5)




In [75]:
split_idx = int(0.9 * len(lines_list))
train_samples = lines_list[:split_idx]
test_samples = lines_list[split_idx:]

val_split_idx = int(0.5 * len(test_samples))
validation_samples = test_samples[:val_split_idx]
test_samples = test_samples[val_split_idx:]

assert len(lines_list) == len(train_samples) + len(test_samples) + len(validation_samples)



In [76]:
len(train_samples)

12017

In [77]:
len(test_samples)

668

In [78]:
len(validation_samples)

668

Data Input Pipelines

In [79]:
base_image_path = os.path.join(base_path, 'lines')
def get_image_paths_and_labels(samples):
  paths = []
  corrected_samples = []
  for (i , file_line) in enumerate(samples):
    line_split = file_line.strip()     ##### Remove spaces at the beginning and end.
    line_split = line_split.split(" ")
    #### Here we provide the image path for the corresponding image in the form like:
    ### part1/part1-part2/part1-part2-part3.png
    image_name = line_split[0]
    partI = image_name.split("-")[0]
    partII = image_name.split("-")[1]
    img_path = os.path.join(
        base_image_path , partI, partI + "-" + partII, image_name + ".png"
    )
    if os.path.getsize(img_path):
      paths.append(img_path)
      corrected_samples.append(file_line.split("\n")[0])
  return paths , corrected_samples


train_img_paths, train_labels = get_image_paths_and_labels(train_samples)
test_img_paths, test_labels = get_image_paths_and_labels(test_samples)
validation_img_paths, validation_labels = get_image_paths_and_labels(validation_samples)

In [80]:
test_img_paths                                                                                                                           

['/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/a03/a03-009/a03-009-02.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/e01/e01-086/e01-086-00.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/d06/d06-027/d06-027-03.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/k02/k02-036/k02-036-01.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/c02/c02-017/c02-017-03.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/d01/d01-056/d01-056-03.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/h04/h04-052/h04-052-02.png',
 '/home/chhabilal/Desktop/treeleaf/Handwritten_Text_Recognition/Handwritten_Text_Recognition/lines/c01/c

In [81]:
test_labels

['a03-009-02 ok 176 22 318 1097 1895 117 in|particular|,|in|Yugoslav|approval|of|the|idea|of|an',
 'e01-086-00 ok 158 41 391 727 1646 77 Two|methods|can|be|used|to|join|the|crochet|to|the|linen|:',
 'd06-027-03 ok 191 20 323 1256 1915 130 be|really|afraid|,|help|us|to|know|that|when',
 'k02-036-01 ok 189 26 368 894 1740 126 young|Lee|protested|,|considerate|as|always|.',
 'c02-017-03 ok 188 27 342 1453 1755 111 table|night|club|near|his|church|be|re-',
 'd01-056-03 ok 189 29 414 1237 1752 117 printed|as|just|over|1,000|different|nouns|,',
 'h04-052-02 err 181 53 396 1291 1666 96 vegetables|,|especially|in|the|first|half|of|the|year|,|although|most',
 'c01-014-09 ok 182 45 293 2515 2032 139 or|at|least|the|most|successfully|executed|trait',
 "r06-097-06 ok 187 18 346 1808 1816 144 a|woman's|.|It|belonged|to|a|Madame",
 'd04-081-07 ok 170 12 415 2007 1673 87 Deliverance|:|Passover|and|its|connection',
 'r06-070-05 ok 173 18 338 1599 1829 189 over|the|Promenade|des|Anglais|.|A|huge',
 'g0

Here we need only words so we remove all others part and find the maximum length and the size of the vocabulary in the training data

In [82]:

train_labels_cleaned = []
characters = set()
max_len = 0

for label in train_labels:
  label = label.split(" ")[-1].strip()
  for char in label:
    characters.add(char)

  max_len = max(max_len , len(label))
  train_labels_cleaned.append(label)

print("Maximum length", max_len)
print("Vocabulary Size", len(characters))


Maximum length 93
Vocabulary Size 79


In [83]:
train_labels_cleaned[0:30]

['is|established|.|Ifor|Huw|Evans|,|Propr.|,',
 'overall|average|for|the|factories|sponsored|by',
 "'|The|younger|men|-|'|'|Which|ones|?|'",
 'with|a|young|and|growing|population|and|excellent',
 "it|#|impaired|the|intellectual|faculties|.|Bunyan's",
 "someone|else|!|'|he|cried|in|a|voice|roughened|by",
 "It|would|now|seem|that|Wicki's|name|must",
 'Against|these|advantages|the|codex|was|fragile|and',
 'The|latter|do|not|regard|themselves|as|expert',
 'for|an|unknown|dependent|or|settle|what|he|regarded',
 'sites|in|Britain|for|the|training|of|German',
 'was|translated|to|Winchester|,|and|Dr.|Hubert',
 'national|income|was|spent|on|social|services|.|The',
 'Bill|is|good|man|,|and|Bueno|Buck|is|raised|on|lake|.',
 '"|in|the|time|of|crisis|it|would|probably|be',
 'people|,|is|good|and|attainable|.|"|We|remain|for|it|.|But|the|Tories',
 'turned|the|conversation|.|"|Let|\'s|ride|over',
 'need|for|the|text|of|the|angel|are|quoted',
 'power|.',
 'The|dinner|table|is|the|best|answer|to|the|gr

Same things done for testing and validation

In [84]:
def clean_labels(samples):
  cleaned_labels = []
  for label in samples:
    label = label.split(" ")[-1].strip()
    cleaned_labels.append(label)
  return cleaned_labels

test_samples_cleaned = clean_labels(test_samples)
validation_samples_cleaned = clean_labels(validation_samples)

In [85]:
validation_samples_cleaned

['been|at|the|Load|of|Trouble|roadhouse|,',
 'with|vertical|members|as|shown|in|the|plans|.|Make|the',
 'keeping|B|flat|on|the|chair|seat|,|and|mark',
 'cut|above|Frank|Haxell|conversationally|.',
 'a|divorce|.|The|judge|says|she|had|to|act|almost',
 'weapons|were|necessary|to|defend|Britain|,|that',
 'largely|based|on|this|.|It|was|on|a',
 'repertory|team|of|comedy|character-actors|jump|through',
 'while|the|worse|for|drink|.|The|family',
 'over|a|question|of|trade|rivalry|.|A|rich|mer-',
 'on|a|river|boat|one|summer|,|to|keep|myself',
 'is|caused|during|the|daytime|,|and|that',
 'one|carrying|a|set|of|small|rollers|.',
 'for|three|years|.|"|I|have|only|2|1/2|years|to|go|before|I|retire|,',
 'ever|heard|.|"|I|\'m|going|to|get|myself',
 'roller-coaster|,|with|occasional|stops|for|an|old',
 'of|moderates|and|Africans|will|reject|the',
 'situation|and|movement|it|matters|a|lot|)|is',
 'he|had|been|subjected|ever|since|the|symptoms|had',
 '3|p.m.|.',
 'it|by|saying|that|the|revision|would

Building the charater Vocabulary

Converting to characters into integers and vice versa

In [86]:
#### Mapping characters to integers
char_to_num = StringLookup(vocabulary = list(characters), mask_token = None)

num_to_char = StringLookup(vocabulary = char_to_num.get_vocabulary(), mask_token = None, invert = True )

Data Preprocessing

Resizing the images without affecting the content of images

In [87]:
def distortion_free_resize(image, img_size):
  w, h = img_size
  image = tf.image.resize(image, size = (h, w), preserve_aspect_ratio = True)

  ### Check amount of padding needed to be done.
  pad_height = h - tf.shape(image)[0]
  pad_width = w - tf.shape(image)[1]

  #### Only necessary if you want to do same amount of padding on both sides.
  if pad_height % 2 != 0:
    height = pad_height // 2
    pad_height_top = height + 1
    pad_height_bottom = height
  else:
    pad_height_top = pad_height_bottom = pad_height // 2

  if pad_width % 2 != 0:
    width = pad_width // 2
    pad_width_left = width + 1
    pad_width_right = width
  else:
    pad_width_left = pad_width_right = pad_width // 2

  image = tf.pad(
      image , 
      paddings = [
                 [pad_height_top , pad_height_bottom],
                 [pad_width_left, pad_width_right],
                 [0,0],
      ],
  )
  image = tf.transpose(image, perm = [1, 0 , 2])
  image = tf.image.flip_left_right(image)
  return image

Putting all the Utilities together

In [88]:
AUTOTUNE = tf.data.AUTOTUNE

batch_size = 64
padding_token = 99
image_width = 128
image_height = 32


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label


def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)



Here we applied all the data preprocessing methods in train , test and validation dataset

In [89]:
train_ds = prepare_dataset(train_img_paths, train_labels_cleaned)
validation_ds = prepare_dataset(validation_img_paths, validation_samples_cleaned)
test_ds = prepare_dataset(test_img_paths, test_samples_cleaned)

In [90]:
test_ds

<PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, None, 1), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

Here this model use CTC loss as an endpoint layer.

In [91]:
class CTCLayer(keras.layers.Layer):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.loss_fn = keras.backend.ctc_batch_cost

  def call(self, y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    loss = self.loss_fn(y_true, y_pred, input_length, label_length)
    self.add_loss(loss)
    return y_pred

Building Model for training

In [92]:
def build_model():
    # Inputs to the model
    input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
    labels = keras.layers.Input(name="label", shape=(None,))

    # First conv block.
    x = keras.layers.Conv2D(32, (3, 3), activation="relu",kernel_initializer="he_normal",padding="same",name="Conv1",)(input_img)
    x = keras.layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Second conv block.
    x = keras.layers.Conv2D(64, (3, 3),activation="relu", kernel_initializer="he_normal",padding="same", name="Conv2",)(x)
    x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)

    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing the output to the RNN part of the model.
    new_shape = ((image_width // 4), (image_height // 4) * 64)
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = keras.layers.Dense(64, activation="relu", name="dense1")(x)
    x = keras.layers.Dropout(0.2)(x)

    # RNNs.
    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # +2 is to account for the two special tokens introduced by the CTC loss.
    # The recommendation comes here: https://git.io/J0eXP.
    x = keras.layers.Dense(len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense2")(x)

    # Add CTC layer for calculating CTC loss at each step.
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model.
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="handwriting_recognizer"
    )
    # Optimizer.
    opt = keras.optimizers.Adam()
    # Compile the model and return.
    model.compile(optimizer=opt)
    return model


# Get the model.
model = build_model()
model.summary()

Model: "handwriting_recognizer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 128, 32, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 32, 32)  320         ['image[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 64, 16, 32)   0           ['Conv1[0][0]']                  
                                                                                                  
 Conv2 (Conv2D)                 (None, 64, 16, 64)   18496       ['pool1[0][0

In [93]:
validation_ds


<PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, None, 1), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

**Evaluation Metrics**

Here we use edit distance as a callback to monitor our model.

We separate the validation images and their labels for convenience.

In [94]:
validation_images = []
validation_labels = []
for i in validation_ds:
    validation_images.append(i['image'])
    validation_labels.append(i['label'])

Now, we create a callback to monitor the edit distances.

In [95]:
def calculate_edit_distance(labels, predictions):
    # Get a single batch and convert its labels to sparse tensors.
    saprse_labels = tf.cast(tf.sparse.from_dense(labels), dtype=tf.int64)

    # Make predictions and convert them to sparse tensors.
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    predictions_decoded = keras.backend.ctc_decode(
        predictions, input_length=input_len, greedy=True
    )[0][0][:, :max_len]
    sparse_predictions = tf.cast(
        tf.sparse.from_dense(predictions_decoded), dtype=tf.int64
    )

    # Compute individual edit distances and average them out.
    edit_distances = tf.edit_distance(
        sparse_predictions, saprse_labels, normalize=False
    )
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []

        for i in range(len(validation_images)):
            labels = validation_labels[i]
            predictions = self.prediction_model.predict(validation_images[i])
            edit_distances.append(calculate_edit_distance(labels, predictions).numpy())

        print(
            f"Mean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}"
        )



Train the model

In [96]:
epochs = 20 
model = build_model()
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
edit_distance_callback = EditDistanceCallback(prediction_model)

# Train the model.
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs,
    callbacks=[edit_distance_callback],
)



Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'handwriting_recognizer/ctc_loss/CTCLoss' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/chhabilal/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 583, in start
      self.io_loop.start()
    File "/home/chhabilal/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/pyth